## How to use 2D velocity estimate functions

This notebook shows how to use `two_dim_velocity_estimates.py` to get a 2D velocity field from either cross-correlation or cross conditional average method

1. estimate_velocity_field() will return a class containing estimation data of all pixels

    movie_data = tdve.estimate_velocity_field(sliced_ds)
    vx = movie_data.get_vx()
    vy = movie_data.get_vy()
    confidences = movie_data.get_confidences()
    R = movie_data.get_R()
    Z = movie_data.get_Z()


2. 

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

import fppanalysis.two_dim_velocity_estimates as tdve

#### Extract estimated velocities

For cross conditional average method the signal must be normalized 

'run_norm_ds' is a function that normalizes the signal

In [ ]:
if method == "cond_av":


    norm_ds = run_norm_ds(sliced_ds, cut_off_freq=1e3)
    movie_data = tdve.estimate_velocity_field(
        norm_ds,
        method,
        min_threshold=min_threshold,
        max_threshold=max_threshold,
        delta=delta,
        window=window,
    )
    vx = movie_data.get_vx()
    vy = movie_data.get_vy()
    confidences = movie_data.get_confidences()
    R = movie_data.get_R()
    Z = movie_data.get_Z()
    events = movie_data.get_events()
else:
    movie_data = tdve.estimate_velocity_field(sliced_ds)
    vx = movie_data.get_vx()
    vy = movie_data.get_vy()
    confidences = movie_data.get_confidences()
    R = movie_data.get_R()
    Z = movie_data.get_Z()